## ABOUT:
- Used fine tuned electra to predict test set
- Observation: 
    - validation loss is not a strong indicator of performance on Kaggle test set
        - electra at 0.41 validation loss performed better than electra at 0.40 validation loss
    - generally model overfits after first epoch
        - in both bert uncased and electra, model trained on 1 epoch performs best in kaggle test set (though it may not have hadthe best validation loss)
    - ELECTRA is performs better than bert uncased
        - even though ELECTRA had higher validation loss, its performance on kaggle test set was better. suggesting a better generalisation 

## load_from_disk

In [1]:
from datasets import load_from_disk
tokenized_test_Dataset = load_from_disk(r"C:\Users\tanch\Documents\Coding Competitions\Kaggle\Natural Language Processing with Disaster Tweets (local)\data\tokenized test set")
tokenized_test_Dataset

Dataset({
    features: ['attention_mask', 'id', 'input_ids', 'new_input_string', 'token_type_ids'],
    num_rows: 3263
})

## AutoModelForSequenceClassification

In [2]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoConfig,DataCollatorWithPadding

In [10]:
model  = AutoModelForSequenceClassification.from_pretrained(r"C:\Users\tanch\Documents\Coding Competitions\Kaggle\Natural Language Processing with Disaster Tweets (local)\notebooks\test-glue\checkpoint-343")

loading configuration file C:\Users\tanch\Documents\Coding Competitions\Kaggle\Natural Language Processing with Disaster Tweets (local)\notebooks\test-glue\checkpoint-343\config.json
Model config ElectraConfig {
  "_name_or_path": "google/electra-base-discriminator",
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.8.2",
  "type_vocab_size": 2,
  "vocab_size": 30522
}

loading weig

In [11]:
print(model.device)
model.to("cuda")

cpu


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

## TrainingArguments and Trainer

In [12]:
args = TrainingArguments(
    "test-glue",
    # evaluation_strategy = "epoch",                           
    # learning_rate=2e-5,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    # num_train_epochs=1,
    # weight_decay=0.01,
    # load_best_model_at_end=True,                               # best model may not be the model at the end of training, thus this param enables us to save any best model during training
    # metric_for_best_model=metric_name,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [13]:
trainer = Trainer(
    model,
    args,
)

## predict

In [14]:
output = trainer.predict(tokenized_test_Dataset)

The following columns in the test set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: id, new_input_string.
***** Running Prediction *****
  Num examples = 3263
  Batch size = 20


In [15]:
y_pred = np.argmax(output.predictions,axis=1)    # axis=1 applies argmax through the columns 

<IPython.core.display.Javascript object>

## export

In [16]:
pd.DataFrame({"id":tokenized_test_Dataset['id'],"target":y_pred}).set_index('id').to_csv(r"C:\Users\tanch\Documents\Coding Competitions\Kaggle\Natural Language Processing with Disaster Tweets (local)\predictions\electra pred 0.csv")

<IPython.core.display.Javascript object>